In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 15: Dueling Generators ---</h2><p>Here, you encounter a pair of dueling <span title="I guess they *are* a little banjo-shaped. Why do you ask?">generators</span>. The generators, called <em>generator A</em> and <em>generator B</em>, are trying to agree on a sequence of numbers. However, one of them is malfunctioning, and so the sequences don't always match.</p>
<p>As they do this, a <em>judge</em> waits for each of them to generate its next value, compares the lowest 16 bits of both values, and keeps track of the number of times those parts of the values match.</p>
<p>The generators both work on the same principle. To create its next value, a generator will take the previous value it produced, multiply it by a <em>factor</em> (generator A uses <code>16807</code>; generator B uses <code>48271</code>), and then keep the remainder of dividing that resulting product by <code>2147483647</code>. That final remainder is the value it produces next.</p>
<p>To calculate each generator's first value, it instead uses a specific starting value as its "previous value" (as listed in your puzzle input).</p>
<p>For example, suppose that for starting values, generator A uses <code>65</code>, while generator B uses <code>8921</code>. Then, the first five pairs of generated values are:</p>
<pre><code>--Gen. A--  --Gen. B--
   1092455   430625591
1181022009  1233683848
 245556042  1431495498
1744312007   137874439
1352636452   285222916
</code></pre>
<p>In binary, these pairs are (with generator A's value first in each pair):</p>
<pre><code>00000000000100001010101101100111
00011001101010101101001100110111

01000110011001001111011100111001
01001001100010001000010110001000

00001110101000101110001101001010
01010101010100101110001101001010

01100111111110000001011011000111
00001000001101111100110000000111

01010000100111111001100000100100
00010001000000000010100000000100
</code></pre>

<p>Here, you can see that the lowest (here, rightmost) 16 bits of the third value match: <code>1110001101001010</code>. Because of this one match, after processing these five pairs, the judge would have added only <code>1</code> to its total.</p>
<p>To get a significant sample, the judge would like to consider <em>40 million</em> pairs. (In the example above, the judge would eventually find a total of <code>588</code> pairs that match in their lowest 16 bits.)</p>
<p>After 40 million pairs, <em>what is the judge's final count</em>?</p>
</article>


In [2]:
from typing import Callable


def gen(factor: int) -> Callable[[int], int]:
    def generator(value: int) -> int:
        return factor * value % 2147483647

    return generator


def judge(a: int, b: int, n=40_000_000) -> int:
    l = 2**16 - 1
    gen_a, gen_b = gen(16807), gen(48271)
    count = 0
    for _ in range(n):
        a = gen_a(a)
        b = gen_b(b)
        if (a ^ b) & l == 0:
            count += 1

    return count


print(f"Example: {judge(65, 8921)} should be 588")

Example: 588 should be 588


In [3]:
# Generator A starts with 873
# Generator B starts with 583
print(f"Part I: {judge(873, 583)}")

Part I: 631


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>631</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>In the interest of trying to align a little better, the generators get more picky about the numbers they actually give to the judge.</p>
<p>They still generate values in the same way, but now they only hand a value to the judge when it meets their <em>criteria</em>:</p>
<ul>
<li>Generator A looks for values that are multiples of <code><em>4</em></code>.</li>
<li>Generator B looks for values that are multiples of <code><em>8</em></code>.</li>
</ul>
<p>Each generator functions completely <em>independently</em>: they both go through values entirely on their own, only occasionally handing an acceptable value to the judge, and otherwise working through the same sequence of values as before until they find one.</p>
<p>The judge still waits for each generator to provide it with a value before comparing them (using the same comparison method as before). It keeps track of the order it receives values; the first values from each generator are compared, then the second values from each generator, then the third values, and so on.</p>
<p>Using the example starting values given above, the generators now produce the following first five values each:</p>
<pre><code>--Gen. A--  --Gen. B--
1352636452  1233683848
1992081072   862516352
 530830436  1159784568
1980017072  1616057672
 740335192   412269392
</code></pre>
<p>These values have the following corresponding binary values:</p>
<pre><code>01010000100111111001100000100100
01001001100010001000010110001000

01110110101111001011111010110000
00110011011010001111010010000000

00011111101000111101010001100100
01000101001000001110100001111000

01110110000001001010100110110000
01100000010100110001010101001000

00101100001000001001111001011000
00011000100100101011101101010000
</code></pre>

<p>Unfortunately, even though this change makes more bits similar on average, none of these values' lowest 16 bits match. Now, it's not until the 1056th pair that the judge finds the first match:</p>
<pre><code>--Gen. A--  --Gen. B--
1023762912   896885216

00111101000001010110000111100000
00110101011101010110000111100000
</code></pre>

<p>This change makes the generators much slower, and the judge is getting impatient; it is now only willing to consider <em>5 million</em> pairs. (Using the values from the example above, after five million pairs, the judge would eventually find a total of <code>309</code> pairs that match in their lowest 16 bits.)</p>
<p>After 5 million pairs, but using this new generator logic, <em>what is the judge's final count</em>?</p>
</article>

</main>


In [4]:
from more_itertools import consume


def gen_II(factor: int, multiple: int) -> Callable[[int], int]:
    def generator(value: int) -> int:
        while True:
            value = factor * value % 2147483647
            if value % multiple == 0:
                return value

    return generator


def judge_II(a: int, b: int, n=5_000_000) -> int:
    l = 2**16 - 1
    gen_a, gen_b = gen_II(16807, 4), gen_II(48271, 8)
    count = 0
    for _ in range(n):
        a = gen_a(a)
        b = gen_b(b)
        if (a ^ b) & l == 0:
            count += 1
    return count


print(f"Example: {judge_II(65, 8921)} should be 309")

Example: 309 should be 309


In [5]:
print(f"Part II: {judge_II(873, 583)}")

Part II: 279


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>279</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


<link href="style.css" rel="stylesheet"></link>
